## Python环境
> 如果需要加载ros-python环境，请将`%%script true`注释。

In [1]:
%%script true
import sys
sys.path.insert(0, '/opt/ros/noetic/lib/python3/dist-packages')


In [2]:
from tf import transformations as tf
import numpy as np

np.set_printoptions(4)

## 旋转矩阵和欧拉角
### 旋转向量和旋转矩阵

In [3]:
alpha, beta, gamma = 0.123, -1.234, 2.345
xaxis, yaxis, zaxis =  (1, 0, 0), (0, 1, 0), (0, 0, 1)

Rx = tf.rotation_matrix(alpha, xaxis) # 旋转向量：围绕x轴旋转alpha度
Ry = tf.rotation_matrix(beta, yaxis) 
Rz = tf.rotation_matrix(gamma, zaxis)
R = tf.concatenate_matrices(Rx, Ry, Rz) # Rx @ Ry @ Rz

p = (1, 1, 1, 0) # 最后的0是齐次坐标

print(R @ p)
print(Rx @ Ry @ Rz @ p)
print(Rx @ (Ry @ (Rz @ p))) # 先绕z，然后y，最后x

[-1.4111  0.1389 -0.9947  0.    ]
[-1.4111  0.1389 -0.9947  0.    ]
[-1.4111  0.1389 -0.9947  0.    ]



### 旋转矩阵转为欧拉角

继续使用上面的`R`：

In [4]:
# 旋转矩阵 => 欧拉角
euler = tf.euler_from_matrix(R, 'rxyz')

np.array([alpha, beta, gamma]), np.array(euler)

(array([ 0.123, -1.234,  2.345]), array([ 0.123, -1.234,  2.345]))

### 旋转矩阵转为欧拉角

In [5]:
Re = tf.euler_matrix(alpha, beta, gamma, 'rxyz')

tf.is_same_transform(R, Re)

True

In [6]:
al, be, ga = tf.euler_from_matrix(Re, 'rxyz')

np.allclose([al, be, ga], [alpha, beta, gamma])

True

## 四元数

假设某个旋转是绕单位向量$\boldsymbol{n} = [nx, ny, nz]^T $进行了角度为$θ$的旋转，那么这个旋转的四元数形式为：

$\boldsymbol{q}=\left[\cos \frac{\theta}{2}, n_{x} \sin \frac{\theta}{2}, n_{y} \sin \frac{\theta}{2}, n_{z} \sin \frac{\theta}{2}\right]^{T}$

在tf python包中，Quaternions $ix+jy+kz+w$ are represented as `[x, y, z, w]`. 

**注意实部`w`在最后，而四元数表示一般将`w`放最前。**

例如绕着x轴$[1, 0, 0]$旋转60度的四元数表示：

In [7]:
angle = np.deg2rad(60) # 60 / 180 * np.pi
axis = (1, 0, 0)
q = tf.quaternion_about_axis(angle, axis)

q

array([0.5  , 0.   , 0.   , 0.866])

通过四元数可以算出旋转轴和旋转角度：

$$
\left\{\begin{array}{l}
\theta=2 \arccos q_{0} \\
{\left[n_{x}, n_{y}, n_{z}\right]^{T}=\left[q_{1}, q_{2}, q_{3}\right]^{T} / \sin \frac{\theta}{2}}
\end{array}\right.
$$

实际使用时需要考虑角度和弧度的转换：

$\theta = 2 \arccos(w) \text{.to\_degree()} $


In [8]:
# np.math.acos(w) * 180 / np.pi * 2
np.rad2deg(np.math.acos(q[3]) * 2)

59.999999999999986

当没有$\theta = 0$时（没有旋转），四元数的虚部都为0，实部为1：

In [9]:
q1 = tf.quaternion_about_axis(np.deg2rad(0), [1, 0, 0])
q2 = tf.quaternion_about_axis(np.deg2rad(0), [2, 0, 0])
q3 = tf.quaternion_about_axis(np.deg2rad(0), [0, 1, 0])

q1, q2, q3

(array([0., 0., 0., 1.]), array([0., 0., 0., 1.]), array([0., 0., 0., 1.]))

### 多次旋转的四元数表示

使用旋转矩阵进行多次旋转时，只要将多个旋转矩阵按顺序进行相乘即可：

In [10]:
alpha, beta, gamma = 0.123, -1.234, 2.345
xaxis, yaxis, zaxis = (1, 0, 0), (0, 1, 0), (0, 0, 1)

Rx = tf.rotation_matrix(alpha, xaxis) # 旋转向量：围绕x轴旋转alpha度
Ry = tf.rotation_matrix(beta, yaxis) 
Rz = tf.rotation_matrix(gamma, zaxis)
R = tf.concatenate_matrices(Rx, Ry, Rz) # Rx @ Ry @ Rz

使用四元数进行多次旋转时，同样将多个四元数按顺序进行相乘：

In [11]:
qx = tf.quaternion_about_axis(alpha, xaxis)
qy = tf.quaternion_about_axis(beta, yaxis)
qz = tf.quaternion_about_axis(gamma, zaxis)

q = tf.quaternion_multiply(qx, qy)
q = tf.quaternion_multiply(q, qz)
Rq = tf.quaternion_matrix(q)

tf.is_same_transform(R, Rq)

True

### 四元数对三维点旋转

四元数可以对三维空间中的点进行旋转。

假设一个空间三维点$\boldsymbol{p}=[x, y, z] \in \mathbb{R}^{3}$，一个由轴角$\boldsymbol{n}, θ$指定的旋转。

三维点$\boldsymbol{p}$经过旋转之后变成为$\boldsymbol{p′}$。如果使用矩阵描述，那么有$\boldsymbol{p′} = \boldsymbol{Rp}$。

用四元数描述旋转：

$$\boldsymbol{p}^{\prime}=\boldsymbol{q} \boldsymbol{p q}^{-1}$$

其中
$\boldsymbol{q}=\left[\cos \frac{\theta}{2}, \boldsymbol{n} \sin \frac{\theta}{2}\right]$

而$\boldsymbol{p}$需要转成四元数表示：

$\boldsymbol{p}=[0, x, y, z]=[0, \boldsymbol{v}]$


先求旋转矩阵对一个点旋转的结果：

In [12]:
# 三维点p=(4,2,3)
p = [4, 2, 3, 0]

# 旋转向量
angle = np.deg2rad(60)
axis = (1, 0, 0)

# 旋转矩阵
R = tf.rotation_matrix(angle, axis)

pt = R @ p

pt

array([ 4.    , -1.5981,  3.2321,  0.    ])

再求四元数对同一个点旋转的结果：

In [13]:
q = tf.quaternion_about_axis(angle, axis)
pq = tf.quaternion_multiply(q, p)
pq = tf.quaternion_multiply(pq, tf.quaternion_inverse(q))

pq

array([ 4.    , -1.5981,  3.2321,  0.    ])